In [10]:
import pandas as pd
import os

# Specify the directory path
directory_path = 'gs://ta-charts-data/origin_data'

# List all files in the directory
all_files = os.listdir(directory_path)

# Filter out files that end with .csv
csv_files = [file for file in all_files if file.endswith('.csv')]

# Read the CSV file
df = pd.read_csv(os.path.join(directory_path, csv_files[0]))
df['datetime'] = pd.to_datetime(df['datetime'])
df


FileNotFoundError: [Errno 2] No such file or directory: 'gs://ta-charts-data/origin_data'

In [17]:
import pandas as pd

#df = pd.read_csv('gs://ta-charts-data/augmented_data/aug_1/*.csv')
df = pd.read_csv('gs://ta-charts-data/origin_data/*.csv')

df['datetime'] = pd.to_datetime(df['datetime'])
df


,index,close,open,high,low,volume,datetime,time
0,57,143.53,143.53,143.53,143.50,275300.0,2000-01-04 09:35:00,0.128205
1,58,143.25,143.56,143.78,143.22,124000.0,2000-01-04 09:40:00,0.256410
2,59,143.56,143.31,143.66,143.22,89200.0,2000-01-04 09:45:00,0.384615
3,60,143.94,143.50,144.06,143.50,67600.0,2000-01-04 09:50:00,0.512821
4,61,143.97,144.00,144.06,143.69,145100.0,2000-01-04 09:55:00,0.641026
...,...,...,...,...,...,...,...,...
469336,469393,473.29,473.56,473.65,473.28,267886.0,2023-12-22 15:40:00,9.487179
469337,469394,473.38,473.29,473.38,472.98,381083.0,2023-12-22 15:45:00,9.615385
469338,469395,473.14,473.38,473.51,473.10,336791.0,2023-12-22 15:50:00,9.743590
469339,469396,473.60,473.12,473.68,473.06,667197.0,2023-12-22 15:55:00,9.871795


In [18]:
#!pip install pandas_market_calendars


In [18]:
# Get market opening days
import pandas as pd 
from pandas_market_calendars import get_calendar
# Get the NYSE calendar
nyse = get_calendar("XNYS")
# Get the valid trading days for the specified date range
trading_days = nyse.valid_days(start_date='2000-01-04', end_date='2023-12-22').date
len(trading_days)

6032

In [20]:
from tqdm import tqdm
from utils.augment_price_data import augment_price_data

# function generates a synthesized data
def obtain_syn_data(original_df:pd.DataFrame, lam:float) -> pd.DataFrame:
    synth_frames = []
    for timestamp in tqdm(trading_days):
        original_df_day = original_df[original_df['datetime'].dt.date == timestamp]
        original_df_day_synth = augment_price_data(original_df_day, lam=lam)
        synth_frames.append(original_df_day_synth)

    # Concatenate all DataFrames in the list
    df_synth = pd.concat(synth_frames, axis=0)

    # Optionally reset the index if needed
    df_synth.reset_index(drop=True, inplace=True)
    # Both have the same dates, volume, time
    df_synth[['datetime', 'volume']] = original_df[['datetime', 'volume']]

    print(f"Augmented: {len(df_synth)}")
    print(f"Original: {len(original_df)}")

    return df_synth

spy_df_synth = obtain_syn_data(df, lam=0.5)
spy_df_synth

100%|██████████| 6032/6032 [21:01<00:00,  4.78it/s]


Augmented: 469341
Original: 469341


,open,high,low,close,datetime,volume
0,143.567956,143.505259,143.457997,143.562341,2000-01-04 09:35:00,275300.0
1,143.534805,143.802619,143.260088,143.237122,2000-01-04 09:40:00,124000.0
2,143.287739,143.632868,143.192299,143.558118,2000-01-04 09:45:00,89200.0
3,143.488380,144.045629,143.476001,143.918082,2000-01-04 09:50:00,67600.0
4,143.985710,144.066612,143.743083,143.966960,2000-01-04 09:55:00,145100.0
...,...,...,...,...,...,...
469336,473.566512,473.675808,473.266268,473.282363,2023-12-22 15:40:00,267886.0
469337,473.332705,473.369441,473.016908,473.424775,2023-12-22 15:45:00,381083.0
469338,473.391041,473.491978,473.067325,473.156083,2023-12-22 15:50:00,336791.0
469339,473.077437,473.674635,473.033121,473.599545,2023-12-22 15:55:00,667197.0


In [21]:
df = pd.read_csv('gs://ta-charts-data/augmented_data/aug_1/*.csv')
#df = pd.read_csv('gs://ta-charts-data/origin_data/*.csv')

df

,open,high,low,close,datetime,volume
0,143.450749,143.528217,143.503627,143.531335,2000-01-04 09:35:00,275300.0
1,143.606006,143.782493,143.207992,143.228975,2000-01-04 09:40:00,124000.0
2,143.321372,143.633865,143.248631,143.588585,2000-01-04 09:45:00,89200.0
3,143.483965,144.079270,143.494439,143.903648,2000-01-04 09:50:00,67600.0
4,143.961262,144.086221,143.716568,143.992883,2000-01-04 09:55:00,145100.0
...,...,...,...,...,...,...
469332,473.757886,473.796491,473.532565,473.739011,2023-12-22 15:20:00,144832.0
469333,473.706167,473.796127,473.484952,473.683710,2023-12-22 15:25:00,271018.0
469334,473.692942,473.991271,473.641165,473.967595,2023-12-22 15:30:00,284534.0
469335,473.944905,473.940221,473.503175,473.524621,2023-12-22 15:35:00,282504.0
